In [1]:
import os

import pickle

import numpy as np
import torch

import open3d as o3d

import matplotlib.pyplot as plt
import matplotlib.cm as cm
cmap = cm.get_cmap('jet')

from tqdm.notebook import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/tmp/ipykernel_1362514/1176535136.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('jet')


In [2]:
import ipywidgets as widgets
from IPython.display import display

In [3]:
PREPROCESSED_3D_DIR = "/home/rsl_admin/openscene/openscene/data/matterport_3d/test"
FUSED_FEATURES_DIR = "/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test"


# PREPROCESSED_3D_DIR = "/home/rsl_admin/openscene/openscene/data/matterport_3d/val"
# FUSED_FEATURES_DIR = "/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg/"


### DO NOT consider running on the training set for now! ###
# Due to weir situation with multiple fused feat files per region


OUTPUT_DIR = "/home/rsl_admin/openscene/comparison_outputs/segmentations"

In [4]:
if "openseg" in FUSED_FEATURES_DIR:
    embedding_space = 'openseg'
else:
    raise Exception("cannot determine the embedding space")
    
split = os.path.basename(PREPROCESSED_3D_DIR)

print(f"embedding space: {embedding_space}\nsplit: {split}")

embedding space: openseg
split: test


##
First check that every 3D region has fused features for that region, pack this data for use later

In [5]:
from collections import namedtuple

In [6]:
RegionDataPaths = namedtuple("RegionDataPaths", ["data_3d_path", "fused_feats_path"])

In [7]:
import re

In [8]:
data_paths = []

for fname in os.listdir(PREPROCESSED_3D_DIR):
    scan_name = fname.split("_")[0]
    
    match = re.search(
        r'_region(\d+)\.pth', 
        fname
    )
    if match:
        region_number = match.group(1)
    else:
        raise RuntimeError(f"No region number for file {fname}")

    
    # Find corresponding fused feature files
    regex = re.compile(f"{scan_name}_region{region_number}_" + r'(\d+)\.pt')
    region_fused_feat_fnames = [
        name for name in os.listdir(FUSED_FEATURES_DIR) if regex.search(name)
    ]
    
    
    if len(region_fused_feat_fnames) == 0:
        print(f"no fused feat files for {fname}, skipping")
        continue
    elif len(region_fused_feat_fnames) > 1:
        print(f"{fname} has multiple fused feat file, this is currently not supported, skipping")
        continue
    
    fused_feat_fname = region_fused_feat_fnames[0]
              
    data_paths.append(
        RegionDataPaths(
            data_3d_path=os.path.join(PREPROCESSED_3D_DIR, fname),
            fused_feats_path=os.path.join(FUSED_FEATURES_DIR, fused_feat_fname)
        )
    )

In [9]:
for dp in data_paths:
    print(dp.data_3d_path)
    print(dp.fused_feats_path)
    print()

/home/rsl_admin/openscene/openscene/data/matterport_3d/test/yqstnuAEVhm_region23.pth
/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test/yqstnuAEVhm_region23_0.pt

/home/rsl_admin/openscene/openscene/data/matterport_3d/test/q9vSo1VnCiC_region13.pth
/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test/q9vSo1VnCiC_region13_0.pt

/home/rsl_admin/openscene/openscene/data/matterport_3d/test/wc2JMjhGNzB_region27.pth
/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test/wc2JMjhGNzB_region27_0.pt

/home/rsl_admin/openscene/openscene/data/matterport_3d/test/5ZKStnWn8Zo_region0.pth
/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test/5ZKStnWn8Zo_region0_0.pt

/home/rsl_admin/openscene/openscene/data/matterport_3d/test/jtcxE69GiFV_region20.pth
/media/rsl_admin/T7/openscene_fused_features/matterport_multiview_openseg_test/jtcxE69GiFV_region20_0.pt

/home/rsl_admin/openscene/openscene/data/matter

## Setup 3D model

In [10]:
'''simple config class to recreate what's done in the openscene code'''
class ModelConfig:
    def __init__(self, feature_2d_extractor, arch_3d):
        self.feature_2d_extractor = feature_2d_extractor
        self.arch_3d = arch_3d

In [11]:
if embedding_space == "openseg":
    checkpoint_path = "/home/rsl_admin/openscene/checkpoints/matterport_openseg.pth"
else:
    raise NotImplementedError

checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage.cuda())

In [12]:
from run.distill import get_model

model_cfg = ModelConfig(
    feature_2d_extractor=embedding_space, 
    arch_3d='MinkUNet18A',
)
model = get_model(model_cfg)
model.load_state_dict(checkpoint['state_dict'], strict=True)

model = model.cuda()

In [13]:
from comparison_utils import DisNetRunner

In [14]:
disnet_runner = DisNetRunner(model)

### Set up class vocabulary
This step requires the user to select a label set to query the segmentations on

In [15]:
labelset_selection_dropdown  = widgets.Dropdown(
    options=[
        "HABITAT_OGN_LABELS",
        "MATTERPORT_REGION_LABELS",
    ],
    disabled=False,
)

display(labelset_selection_dropdown)

Dropdown(options=('HABITAT_OGN_LABELS', 'MATTERPORT_REGION_LABELS'), value='HABITAT_OGN_LABELS')

In [16]:
if labelset_selection_dropdown.value == "HABITAT_OGN_LABELS":
    from comparison_label_mappings import HABITAT_OGN_LABELS, HABITAT_OGN_LABELS_TO_TEXT_PROMPTS
    
    labelset = list(HABITAT_OGN_LABELS)
    label_text_prompts = [
        HABITAT_OGN_LABELS_TO_TEXT_PROMPTS[label] for label in HABITAT_OGN_LABELS
    ]
    
    
elif labelset_selection_dropdown.value == "MATTERPORT_REGION_LABELS":
    from comparison_label_mappings import MATTERPORT_REGION_LABELS, MATTERPORT_REGION_LABELS_TO_TEXT_PROMPTS
    
    labelset = list(MATTERPORT_REGION_LABELS)
    label_text_prompts = [
        MATTERPORT_REGION_LABELS_TO_TEXT_PROMPTS[label] for label in MATTERPORT_REGION_LABELS
    ]
    
else: 
    raise Exception


In [17]:
label_to_ind = {label: i for i, label in enumerate(labelset)}

print(label_to_ind)

{'bathroom': 0, 'bedroom': 1, 'closet': 2, 'dining room': 3, 'garage': 4, 'hallway': 5, 'library': 6, 'laundryroom/mudroom': 7, 'kitchen': 8, 'living room': 9, 'meetingroom/conferenceroom': 10, 'office': 11, 'porch/terrace/deck/driveway': 12, 'rec/game': 13, 'stairs': 14, 'utilityroom/toolroom': 15, 'tv': 16, 'workout/gym/exercise': 17, 'balcony': 18, 'bar': 19, 'classroom': 20, 'spa/sauna': 21, 'entryway/foyer/lobby': 22, 'outdoor': 23, 'dining booth': 24, 'other room': 25}


In [18]:
from comparison_utils import extract_text_feature

In [19]:
label_feats = extract_text_feature(
    label_text_prompts,
)

print(label_feats.shape)

Loading CLIP ViT-L/14@336px model...
Finish loading
torch.Size([26, 768])


## Generate and save the segmentations

In [20]:
save_dirname = f"{embedding_space}-matterport-{split}"

os.makedirs(
    os.path.join(OUTPUT_DIR, save_dirname),
    exist_ok=True
)

print(f"saving to: {os.path.join(OUTPUT_DIR, save_dirname)}")

saving to: /home/rsl_admin/openscene/comparison_outputs/segmentations/openseg-matterport-test


In [21]:
from comparison_utils import get_region_features, compute_predictions

In [22]:
for dp in tqdm(data_paths):
    
    points, colors, fused_feats, distill_feats = get_region_features(
        dp.data_3d_path,
        dp.fused_feats_path,
        disnet_runner,
    )
    
    preds = compute_predictions(
        label_feats,
        fused_feats,
        distill_feats,
        method="ensemble"
    )
    
    out = {
        "points": points.astype(np.float16),
        "colors": colors.astype(np.float16),
        "preds": preds.astype(np.int16),
        "label_to_ind": label_to_ind,
    }
    
    save_filename = os.path.basename(dp.data_3d_path).replace('.pth', '.pkl')
    save_path = os.path.join(
        os.path.join(OUTPUT_DIR, save_dirname, save_filename),
    )
    
    with open(save_path, 'wb') as f:
        pickle.dump(out, f)

  0%|          | 0/406 [00:00<?, ?it/s]

## Visualize an output segmentation

In [23]:
MATTERPORT_DIR = "/media/rsl_admin/T7/matterport/data/v1/scans"

In [24]:
np.random.seed(0)

In [25]:
output_fname = np.random.choice(
    os.listdir(os.path.join(OUTPUT_DIR, save_dirname))
)

# output_fname = "2t7WUuJeko7_region0.pkl"

output_path = os.path.join(OUTPUT_DIR, save_dirname, output_fname)

with open(output_path, 'rb') as f:
    out = pickle.load(f)
    
print(f"loaded {output_fname}")

loaded 2t7WUuJeko7_region3.pkl


In [26]:
scan_name = output_fname.split('_')[0]

scan_mesh = o3d.io.read_triangle_mesh(
    os.path.join(MATTERPORT_DIR, f"{scan_name}/{scan_name}/house_segmentations/{scan_name}.ply")
)

In [27]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(out["points"])
pcd.colors = o3d.utility.Vector3dVector(out["colors"])

In [28]:
label_to_ind = out["label_to_ind"]
labels = label_to_ind.keys()

In [29]:
print(labels)

dict_keys(['bathroom', 'bedroom', 'closet', 'dining room', 'garage', 'hallway', 'library', 'laundryroom/mudroom', 'kitchen', 'living room', 'meetingroom/conferenceroom', 'office', 'porch/terrace/deck/driveway', 'rec/game', 'stairs', 'utilityroom/toolroom', 'tv', 'workout/gym/exercise', 'balcony', 'bar', 'classroom', 'spa/sauna', 'entryway/foyer/lobby', 'outdoor', 'dining booth', 'other room'])


In [30]:
label_dropdown = widgets.Dropdown(
    options=['all'] + sorted(labels),
    disabled=False,
)

display(label_dropdown)

Dropdown(options=('all', 'balcony', 'bar', 'bathroom', 'bedroom', 'classroom', 'closet', 'dining booth', 'dini…

In [44]:
label_inds = np.where(out["preds"] == label_to_ind[label_dropdown.value])[0]

o3d.visualization.draw_geometries(
    [
        scan_mesh, 
        pcd.select_by_index(label_inds).paint_uniform_color((0,1,0))
    ]
)